Start out by establishing a connection to a MongoDB instance.

In [76]:
import pymongo
conn = pymongo.MongoClient()
db = conn.test
coll = db.objects
coll.drop()

Insert a couple of documents that represent themselves naturally in JSON.

In [77]:
import datetime
coll.insert({'a': datetime.datetime.now(), 'b': 1.0})
coll.insert({'items': [1, 2, 3, 'd']})

ObjectId('52e523b112d4aa11ec2a43e6')

But what about more complex objects (classes and instances of those classes)?

In [78]:
class MyObject:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return '{self.__class__.__name__}({self.x}, {self.y})'.format(**vars())

ob1 = MyObject(1, 2)
ob2 = MyObject(0, 5)

In [79]:
coll.insert(ob1)

TypeError: 'MyObject' object is not iterable

In [80]:
import jaraco.modb
jaraco.modb.encode(ob1)

{'py/object': '__main__.MyObject', 'x': 1, 'y': 2}

Because a MyObject instance doesn't have a natural representation in JSON, it's serialized as a dictionary with a special key 'py/object', which signals to the decoder that this is a JSONPickled Python Object. As long as the system doing the decoding implements `__main__.MyObject` with a compatible interface, the object will decode nicely.

In [81]:
coll.insert(jaraco.modb.encode(ob1))
coll.insert(jaraco.modb.encode(ob2))

ObjectId('52e523b112d4aa11ec2a43e8')

Now the two objects should be persisted to the database. Query them to see how they appear.

In [82]:
list(coll.find())

[{'_id': ObjectId('52e523b112d4aa11ec2a43e5'),
  'a': datetime.datetime(2014, 1, 26, 10, 3, 13, 522000),
  'b': 1.0},
 {'_id': ObjectId('52e523b112d4aa11ec2a43e6'), 'items': [1, 2, 3, 'd']},
 {'_id': ObjectId('52e523b112d4aa11ec2a43e7'),
  'py/object': '__main__.MyObject',
  'x': 1,
  'y': 2},
 {'_id': ObjectId('52e523b112d4aa11ec2a43e8'),
  'py/object': '__main__.MyObject',
  'x': 0,
  'y': 5}]

In [83]:
next(map(jaraco.modb.decode, coll.find({'x': 0})))

MyObject(0, 5)

But what about more complex objects? Consider ob3 whose x attribute is another MyObject.

In [84]:
ob3 = MyObject(ob2, 2)
ob3

MyObject(MyObject(0, 5), 2)

In [85]:
coll.insert(jaraco.modb.encode(ob3))

ObjectId('52e523b112d4aa11ec2a43e9')

Because MongoDB's document query engine allows reaching deep into the documents, one can even query based on child object's attributes.

In [86]:
# Find all objects whose x attribute has a y attribute with a value of 5
query = {'x.y': 5}
next(map(jaraco.modb.decode, coll.find(query)))

MyObject(MyObject(0, 5), 2)

Where are the limitations? What about integer keys?

In [87]:
coll.insert({1: 3})

InvalidDocument: documents must have only string keys, key was 1

In [93]:
id = coll.insert(jaraco.modb.encode({1: 3}))

In [97]:
coll.find_one({'_id': id})

{'1': 3, '_id': ObjectId('52e5245e12d4aa11ec2a43ec')}

You might note that the integer 1 is now represented as a string '1'. This limitation is an unfortunate side-effect of relying on JSON as a serialization layer.